# Training fine flow prediction
Assuming source image $I_s$ and target image $I_t$ are already coarsely aligned, this notebook will try to predict a fine flow $F_{s\rightarrow t}$ between them. 

TODO describe objective functions used in this project

In [1]:
%load_ext autoreload
%autoreload 2

We assume you already have a zipped dataset in `data` folder.

In [2]:
%cd workspace
!ln -s ../data/MegaDepth_cleansed.zip MegaDepth_cleansed.zip

/mnt/d/courses/RANSAC-Flow/notebooks/workspace


If you are working in Google Colab, you might find this cell useful. 
It performs 

0. Sanity check if you are using Google Colab
1. Mount Google Drive. 
2. Assume you have a folder `RANSAC-Flow` that is equivalent to this repository, which contains `data` folder.
3. Copy the dataset to `/tmp` 

In [6]:
# 0.
try:
    import google.colab
    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False
finally:
    if not IN_COLAB:
        raise RuntimeError('not running on Google Colab')

# 1.
from google.colab import drive
drive.mount('/content/drive')

# 2. 
%cd /content/drive/MyDrive/RANSAC-Flow

# 3.
!rsync -ah --progress data/MegaDepth_cleansed.zip /tmp
!ln -s /tmp/MegaDepth_cleansed.zip MegaDepth_cleansed.zip

RuntimeError: not running on Google Colab

Import packages that we will use throughout this notebook.

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

We enable logging here to make debug easier. 

In [4]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(name)s :: %(levelname)s :: %(message)s",
    handlers=[logging.StreamHandler()],
)

# fine tune submodules log level
logging.getLogger("ransacflow").setLevel(logging.WARNING)
logging.getLogger('ransacflow.data').setLevel(logging.WARNING)

## Prepare dataset
We already pack some datasets used in the original paper as `LightningDataModule`. We will import it here.

In [5]:
from ransacflow.data import MegaDepthDataModule

mega_depth = MegaDepthDataModule(
    "MegaDepth_cleansed.zip",
    train_image_size=224,
    train_batch_size=2,
    val_image_size=480,
)

In [ ]:
#TODO add some sanity check for the dataset here, previews

In [ ]:
# TODO setup environments for the following training sessions, how?

In [ ]:
# FIXME is it possible to share the Trainer object across all 3 stages

## Stage 1
Only train the **reconstruction loss**. 

It is based on the idea that source image $I_s$ warped with the predicted flow $F_{s\rightarrow t}$ should align well with the target image $I_t$. In the original work, they use the structural similarity (SSIM) as the perception model. 
$$ L_{\text{recon}}\left(I_s, I_t\right) = \sum_{(x,y)\in I_t} M_t^{\text{cycle}}(x,y) \left( 1 - \text{SSIM}\left\lbrace I_s(x^\prime, y^\prime), I_t(x,y) \right\rbrace \right) $$

FIXME wtf is M_t doing here?

In [5]:
# DEBUG somehow dataset needs to reload everythime for correct coords
from ransacflow.data import MegaDepthDataModule

mega_depth = MegaDepthDataModule(
    "MegaDepth_cleansed.zip",
    train_image_size=224,
    train_batch_size=4,
    val_image_size=480,
    num_workers=8
)

## parameter names

log_dir = "MegaDepth_logs"

##

from ransacflow.train import RANSACFlowModelStage1

ransac_flow = RANSACFlowModelStage1(
    alpha=0, beta=0, gamma=0, kernel_size=7, ssim_window_size=11, lr=2e-4,
)

# FIXME unify TB logging location and experiment name
trainer = Trainer(
    gpus=-1,
    fast_dev_run=False,
    max_epochs=200,
    val_check_interval=0.25,
    logger=TensorBoardLogger(log_dir, name="stage1"),
    callbacks=[EarlyStopping(monitor="val_loss", min_delta=0.01, patience=3)],
)
trainer.fit(ransac_flow, mega_depth)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
[2021-12-28 01:56:45,456] ransacflow.data.megadepth :: WARNING :: file integrity is not explicitly tested
[2021-12-28 01:56:46,812] ransacflow.data.megadepth :: WARNING :: file integrity is not explicitly tested
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                  | Params
------------------------------------------------------------
0 | feature_extractor | FeatureExtractor      | 2.8 M 
1 | correlator        | NeighborCorrelator    | 0     
2 | flow              | FlowPredictor         | 1.8 M 
3 | matchability      | MatchabilityPredictor | 1.7 M 
4 | loss_rec          | ReconstructionLoss    | 0     
------------------------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.948    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/andy/.conda/envs/ransacflow/lib/python3.8/site-packages/torch/nn/functional.py:4065: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/andy/.conda/envs/ransacflow/lib/python3.8/site-packages/torch/nn/functional.py:4003: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/andy/.conda/envs/ransacflow/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


prec@1.0=0.01299
prec@2.0=0.02374
prec@3.0=0.06696
prec@5.0=0.16304
prec@8.0=0.27748
prec@13.0=0.47905
prec@22.0=0.82621
prec@36.0=1.00000


Training: 0it [00:00, ?it/s]

/home/andy/.conda/envs/ransacflow/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


All following command line interface are copied from the original implementation, temporarily.

In [ ]:
    --nEpochs 200 
    --lr 2e-4
    --kernelSize 7 
--imgSize 224 
--batchSize 16 
    --lambda-match 0.0, alpha 
    --mu-cycle 0.0, beta 
    --grad 0.0, gamma  
    --trainMode flow 
--margin 88 

## Stage 2
Train jointly the **reconstruction loss** and **cycle consistency of the flow**.

Asides from the reconstruction loss mentioned in previous stage, we start to enforce cycle consistency of the flow by
$$ L_{\text{cycle}} = \sum_{(x,y) \in I_t} M_t^{\text{circle}} (x,y) \left\lVert \left(x^\prime, y^\prime \right), \bm{F}_{t\rightarrow s}(x,y) \right\rVert_2 $$

FIXME what happened with (x^\prime, y^\prime), F_{t->s}? Are they multiplied?

In [ ]:
from ransacflow.train import RANSACFlowModelStage2

ransac_flow = RANSACFlowModelStage2(alpha=0, beta=1, gamma=0, kernel_size=7, lr=2e-4)

# FIXME unify TB logging location and experiment name
trainer = Trainer(
    max_epochs=50,
    logger=TensorBoardLogger("tb_logs", name="RANSAC-Flow_stage2"),
    callbacks=[EarlyStoppping(monitor="val_loss", min_delta=0.01, patience=3)],
)
trainer.fit(ransac_flow, MegaDepthDataModule)

In [ ]:

    --nEpochs 50 
    --lr 2e-4 
    --kernelSize 7 
--imgSize 224 
--batchSize 16 
    --lambda-match 0.0, alpha
    --mu-cycle 1.0, beta
    --grad 0.0, gamma
    --trainMode flow 
--margin 88 

## Stage 3
Train all three losses together: **reconstruction loss**, **cycle consistency of the flow**, and **matchability loss**.

Matchability mask can be seen as pixel-wise weights for the reconstruction and cycle consistency loss. These losses encourage th matchability to be zero. To counteract this effect, the matchability loss encourages the matchability mask to be close to one.

FIXME equation for matchability
FIXME still doesn't understand what matchability actually implies, what is the difference between this and cycle loss?

In [ ]:
from ransacflow.train import RANSACFlowModelStage3

ransac_flow = RANSACFlowModelStage3(alpha=0.01, beta=1, gamma=0, kernel_size=7, lr=2e-4)

# FIXME unify TB logging location and experiment name
trainer = Trainer(
    max_epochs=50,
    logger=TensorBoardLogger("tb_logs", name="RANSAC-Flow_stage3"),
    callbacks=[EarlyStoppping(monitor="val_loss", min_delta=0.01, patience=3)],
)
trainer.fit(ransac_flow, MegaDepthDataModule)


In [ ]:
    --nEpochs 50 
    --lr 2e-4
    --kernelSize 7 
--imgSize 224 
--batchSize 16 
    --lambda-match 0.01, alpha
    --mu-cycle 1.0, beta
    --grad 0.0, gamma
    --trainMode flow+match 
--margin 88 


## Stage 4.1
This additional stage fine tune on SOMETHING MAGICAL, so the output image introduce less distortions.

TODO need to update description from the original paper

## Stage 4.2
This additional stage uses perceptual loss, 

TODO add description about why and how to use perceptual loss